# Sparse Autoencoder Training Demo

This demo trains a sparse autoencoder on activations from a Tiny Stories 1M model.

To do this we setup a *source model* (the TinyStories model) that we want to generate activations
from, along with a *source dataset* of prompts to help generate these activations.

We also setup a *sparse autoencoder model* which we'll train on these generated activations, to
learn a sparse representation of them in higher dimensional space.

Finally we'll wrap this all together in a *pipeline*, which alternates between generating
activations (storing them in ram), and training the SAE on said activations.

## Setup

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

import torch
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import PreTrainedTokenizerBase
import wandb

from sparse_autoencoder import SparseAutoencoder
from sparse_autoencoder.activation_resampler import ActivationResampler
from sparse_autoencoder.loss.learned_activations_l1 import LearnedActivationsL1Loss
from sparse_autoencoder.loss.mse_reconstruction_loss import MSEReconstructionLoss
from sparse_autoencoder.loss.reducer import LossReducer
from sparse_autoencoder.optimizer.adam_with_reset import AdamWithReset
from sparse_autoencoder.source_data.text_dataset import GenericTextDataset
from sparse_autoencoder.train.pipeline import Pipeline


os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = get_device()
print(f"Using device: {device}")  # You will need a GPU

Using device: mps


### Hyperparameters

The way this library works is that you can define your own hyper-parameters and then setup the
underlying components with them. This is extremely flexible, but to help you get started we've
included some common ones below along with some sensible defaults. You can also easily sweep through
multiple hyperparameters with `wandb.sweep`.

In [2]:
hyperparameters = {
    # Expansion factor is the number of features in the sparse representation, relative to the
    # number of features in the original MLP layer. The original paper experimented with 1x to 256x,
    # and we have found that 4x is a good starting point.
    "expansion_factor": 4,
    # L1 coefficient is the coefficient of the L1 regularization term (used to encourage sparsity).
    "l1_coefficient": 0.001,
    # Adam parameters (set to the default ones here)
    "lr": 0.001,
    "adam_beta_1": 0.9,
    "adam_beta_2": 0.999,
    "adam_epsilon": 1e-8,
    "adam_weight_decay": 0.0,
    # Batch sizes
    "train_batch_size": 8192,
}

### Source Model

The source model is just a [TransformerLens](https://github.com/neelnanda-io/TransformerLens) model
(see [here](https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
for a full list of supported models).

In this example we're training a sparse autoencoder on the activations from the first MLP layer, so
we'll also get some details about that hook point.

In [3]:
# Source model setup with TransformerLens
src_model_name = "tiny-stories-1M"
src_model = HookedTransformer.from_pretrained(src_model_name, dtype="float32")

# Details about the activations we'll train the sparse autoencoder on
src_model_activation_hook_point = "blocks.0.mlp.hook_post"
src_model_activation_layer = 0
src_d_mlp: int = src_model.cfg.d_mlp  # type: ignore (TransformerLens typing is currently broken)

f"Source: {src_model_name}, Hook: {src_model_activation_hook_point}, Features: {src_d_mlp}"

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-1M into HookedTransformer


'Source: tiny-stories-1M, Hook: blocks.0.mlp.hook_post, Features: 256'

### Sparse Autoencoder

We can then setup the sparse autoencoder. The default model (`SparseAutoencoder`) is setup as per
the original Anthropic paper [Towards Monosemanticity: Decomposing Language Models With Dictionary
Learning ](https://transformer-circuits.pub/2023/monosemantic-features/index.html).

However it's just a standard PyTorch model, so you can create your own model instead if you want to
use a different architecture. To do this you just need to extend the `AbstractAutoencoder`, and
optionally the underlying `AbstractEncoder`, `AbstractDecoder` and `AbstractOuterBias`. See these
classes (which are fully documented) for more details.

In [4]:
expansion_factor = hyperparameters["expansion_factor"]
autoencoder = SparseAutoencoder(
    n_input_features=src_d_mlp,  # size of the activations we are autoencoding
    n_learned_features=int(src_d_mlp * expansion_factor),  # size of SAE
    geometric_median_dataset=torch.zeros(src_d_mlp),  # this is used to initialize the tied bias
).to(device)
autoencoder  # Print the model (it's pretty straightforward)

SparseAutoencoder(
  (_pre_encoder_bias): TiedBias(position=pre_encoder)
  (_encoder): LinearEncoder(
    in_features=256, out_features=1024
    (activation_function): ReLU()
  )
  (_decoder): UnitNormDecoder(in_features=1024, out_features=256)
  (_post_decoder_bias): TiedBias(position=post_decoder)
)

We'll also want to setup an Optimizer and Loss function. In this case we'll also use the standard
approach from the original Anthropic paper. However you can create your own loss functions and
optimizers by extending `AbstractLoss` and `AbstractOptimizerWithReset` respectively.

In [5]:
# We use a loss reducer, which simply adds up the losses from the underlying loss functions.
loss = LossReducer(
    LearnedActivationsL1Loss(
        l1_coefficient=hyperparameters["l1_coefficient"],
    ),
    MSEReconstructionLoss(),
)
loss

LossReducer(
  (0): LearnedActivationsL1Loss(l1_coefficient=0.001)
  (1): MSEReconstructionLoss()
)

In [6]:
optimizer = AdamWithReset(
    params=autoencoder.parameters(),
    named_parameters=autoencoder.named_parameters(),
    lr=hyperparameters["lr"],
    betas=(hyperparameters["adam_beta_1"], hyperparameters["adam_beta_2"]),
    eps=hyperparameters["adam_epsilon"],
    weight_decay=hyperparameters["adam_weight_decay"],
)
optimizer

AdamWithReset (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)

Finally we'll initialise an activation resampler.

In [7]:
activation_resampler = ActivationResampler()

### Source dataset

This is just a dataset of tokenized prompts, to be used in generating activations (which are in turn
used to train the SAE).

In [8]:
tokenizer: PreTrainedTokenizerBase = src_model.tokenizer  # type: ignore
source_data = GenericTextDataset(tokenizer=tokenizer, dataset_path="roneneldan/TinyStories")

/Users/alan/Documents/Repos/sparse_autoencoder/.venv/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to
wandb. However, we should pass in a dictionary with all of our hyperaparameters so they're on 
wandb. 

We strongly encourage users to make use of wandb in order to understand the training process.

In [19]:
Path(".cache/").mkdir(exist_ok=True)
wandb.init(
    project="sparse-autoencoder",
    dir=".cache",
    config=hyperparameters,
)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LearnedActivationsL1Loss,▃▅▆█▃▇▇██▅▅▆▄▂▃▂▂▃▃▃▄▂▁▃▄▄
LossReducer,▇▆▇▇▇█▄▇▇▆▆▅▁▄▇▃▅▄▅▅▄▃▄▅▂▁
MSEReconstructionLoss,▇▆▇▇▇█▄▇▇▆▆▅▁▄▇▄▅▄▅▅▄▃▄▅▂▁
LearnedActivationsL1Loss,0.00015
LossReducer,0.01652
MSEReconstructionLoss,0.01637


In [24]:
pipeline = Pipeline(
    cache_name=src_model_activation_hook_point,
    layer=src_model_activation_layer,
    source_model=src_model,
    autoencoder=autoencoder,
    source_dataset=source_data,
    optimizer=optimizer,
    loss=loss,
    activation_resampler=activation_resampler,
    source_data_batch_size=8,
    checkpoint_directory=Path("../../.checkpoints"),
)

pipeline.run_pipeline(
    train_batch_size=int(hyperparameters["train_batch_size"]),
    max_store_size=100_000,
    # Sizes for demo purposes (you probably want to scale these by 10x)
    max_activations=10_000_000,
    resample_frequency=100_000,
    checkpoint_frequency=100_000,
)

Activations trained on:   0%|          | 0/10000000 [00:00<?, ?it/s]

wandb: WARNING (User provided step: 1 is less than current step: 26. Dropping entry: {'LearnedActivationsL1Loss': 0.00014641458983533084, 'MSEReconstructionLoss': 0.016486823558807373, 'LossReducer': 0.01663323864340782, '_timestamp': 1700695156.0421581}).
wandb: WARNING (User provided step: 2 is less than current step: 26. Dropping entry: {'LearnedActivationsL1Loss': 0.00014898329391144216, 'MSEReconstructionLoss': 0.016573363915085793, 'LossReducer': 0.01672234758734703, '_timestamp': 1700695156.200486}).
wandb: WARNING (User provided step: 3 is less than current step: 26. Dropping entry: {'LearnedActivationsL1Loss': 0.00014914142957422882, 'MSEReconstructionLoss': 0.016520731151103973, 'LossReducer': 0.016669873148202896, '_timestamp': 1700695156.23089}).
wandb: WARNING (User provided step: 4 is less than current step: 26. Dropping entry: {'LearnedActivationsL1Loss': 0.00014537939568981528, 'MSEReconstructionLoss': 0.016562525182962418, 'LossReducer': 0.016707904636859894, '_timesta

RuntimeError: shape mismatch: value tensor of shape [846, 256] cannot be broadcast to indexing result of shape [1024, 846]

In [ ]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LearnedActivationsL1Loss,█▇▆▆▆▆▇▆▆▆▇▇▅▅▄▅▄▅▆▅▄▄▄▃▃▃▂▄▆▄▆▃▃▁▂▄▃▄▄▅
LossReducer,▆▆▄▆▄▅▁▅▆▆▅▇▆▆▄▄▆▆▇█▆▆▆▂▅▆▅▄▅▇▆▆▇▄▄▅▆▅▅▃
MSEReconstructionLoss,▆▆▄▆▄▅▁▅▆▆▅▇▆▆▄▄▆▆▇█▆▆▆▃▅▆▆▄▅▇▆▆▇▄▄▆▆▅▅▃
LearnedActivationsL1Loss,0.00015
LossReducer,0.01636
MSEReconstructionLoss,0.01621


## Training Advice

-- Unfinished --

- Check recovery loss is low while sparsity is low as well (<20 L1) usually.
- Can't be sure features are useful until you dig into them more. 

# Analysis

-- Unfinished --